In [1]:
# import functions from a file stored in a different directory
import sys
sys.path.append("/home/ubuntu/work/therapeutic_accelerator/scripts/utils")
sys.path.append("/home/ubuntu/work/therapeutic_accelerator/scripts/database")



In [3]:
import chromaDB_connection as cDBc
from db_tools import db_connection
from utils import import_config

config, keys = import_config()

# engine = db_connection(password = keys["postgres"], host = config["database"]["host"])

# Chroma setup
chroma_server_host = "52.23.195.129"
chroma_client = cDBc.create_chroma_client(chroma_server_host)

# Working Collection
collection = chroma_client.get_or_create_collection("specter_abstracts")

Nanosecond heartbeat on server 1689863655921779057000


[Collection(name=langchain_store),
 Collection(name=abstracts),
 Collection(name=fulltext),
 Collection(name=specter_abstracts)]

In [4]:
question = "What is pancreatic cancer?"

results = cDBc.query_chroma(collection, question)

context = ''.join(results['documents'])

In [5]:
results['documents']

['Microvascular density (MVD), a marker for tumor angiogenesis, has been demonstrated to have prognostic significance in various malignancies. Previous studies have demonstrated that MVD is an independent prognostic factor in pancreatic adenocarcinoma and that longer survival is associated with hypovascular tumors. The prognostic importance of MVD in pancreatic neuroendocrine tumor (NET) has not been documented. We evaluated MVD in pancreatic NET and correlated it with clinicopathologic features and patient outcome to determine whether MVD is a useful prognostic indicator for these patients. Twenty-five pancreatic NETs from our archival files resected between 1981 and 2000 were identified. The mean MVD was determined for each tumor from the 3 most vascularized 200 × fields. Clinical follow-up ranged from 1 to 19 years, with a mean of 4.9 years. At last follow-up, 6 patients were dead of disease, 10 patients were alive without disease, 4 patients were alive with disease, and 5 patients 

In [2]:
prompts = pd.read_csv(
    '/home/ubuntu/work/therapeutic_accelerator/data/prompts.csv')
# testing prompt one
question = prompts.loc[0, "Prompt"]

# Load LED model


In [82]:
from transformers import AutoTokenizer, LongformerForQuestionAnswering
import torch

led_tokenizer = AutoTokenizer.from_pretrained(
    "allenai/longformer-large-4096-finetuned-triviaqa")
led_model = LongformerForQuestionAnswering.from_pretrained(
    "allenai/longformer-large-4096-finetuned-triviaqa")

In [ ]:
text = "".join(results["documents"][0])

In [ ]:
encoding = led_tokenizer(question, text, return_tensors="pt")

input_ids = encoding["input_ids"]

In [ ]:
# default is local attention everywhere
# the forward method will automatically set global attention on question tokens
attention_mask = encoding["attention_mask"]

outputs = led_model(input_ids, attention_mask=attention_mask)

In [ ]:
start_logits = outputs.start_logits
end_logits = outputs.end_logits

all_tokens = led_tokenizer.convert_ids_to_tokens(input_ids[0].tolist())

answer_tokens = all_tokens[torch.argmax(
    start_logits): torch.argmax(end_logits) + 1]

answer = led_tokenizer.decode(
    led_tokenizer.convert_tokens_to_ids(answer_tokens)
)  # remove space prepending space token

In [6]:
documents = text_splitter.create_documents(results["documents"][0])
documents

[Document(page_content='Microvascular density (MVD), a marker for tumor angiogenesis, has been demonstrated to have prognostic significance in various malignancies. Previous studies have demonstrated that MVD is an independent prognostic factor in pancreatic adenocarcinoma and that longer survival is associated with hypovascular tumors. The prognostic importance of MVD in pancreatic neuroendocrine tumor (NET) has not been documented. We evaluated MVD in pancreatic NET and correlated it with clinicopathologic features and patient outcome to determine whether MVD is a useful prognostic indicator for these patients. Twenty-five pancreatic NETs from our archival files resected between 1981 and 2000 were identified. The mean MVD was determined for each tumor from the 3 most vascularized 200 × fields. Clinical follow-up ranged from 1 to 19 years, with a mean of 4.9 years. At last follow-up, 6 patients were dead of disease, 10 patients were alive without disease, 4 patients were alive with di

In [7]:
class specter_ef(EmbeddingFunction):
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def embed_documents(self, texts: Documents) -> Embeddings:

        text_list = [re.sub("\n", " ", p) for p in texts]
        texts = [re.sub("\s\s+", " ", t) for t in text_list]

        # embed the documents somehow
        embeddings = []

        for text in texts:
            inputs = tokenizer(text, padding=True, truncation=True,
                               return_tensors="pt", max_length=512)
            result = model(**inputs)
            embeddings.append(result.last_hidden_state[:, 0, :])

        return embeddings


specter_embeder = specter_ef(model, tokenizer)

In [18]:
# remove stop words and punctuation from a string using the nltk library
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [34]:
test_string = 'Background: Recurrent laryngeal nerve paralysis (RLNP), a severe complication of mini-invasive esophagectomy, usually occurs during lymphadenectomy adjacent to recurrent laryngeal nerve. '

In [ ]:
# remove stop words from test_string
stop_words = set(stopwords.words("english"))
from string import punctuation

exclude_punctuation = set(punctuation)
# combine stop words and punctuation marks
exclude = set.union(stop_words, exclude_punctuation)

In [35]:
# use stop_words to remove stop words from test_string
test_string = ' '.join(
    [word for word in test_string.split() if word not in exclude])

In [61]:
# service_context = ServiceContext.from_defaults(llm = led_model, chunk_size=512)

NameError: name 'led_model' is not defined

In [52]:
from llama_index import Document

In [59]:
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, service_context=service_context
)

AttributeError: 'Document' object has no attribute 'get_doc_id'

In [ ]:
# Query Data
query_engine = index.as_query_engine()

response = query_engine.query("What did the author do growing up?")

display(Markdown(f"<b>{response}</b>"))

In [85]:
from llama_index import download_loader

ChromaReader = download_loader("ChromaReader")

In [87]:
help(ChromaReader)

Help on class ChromaReader:

class ChromaReader(llama_index.readers.base.BaseReader)
 |  ChromaReader(collection_name: str, persist_directory: str) -> None
 |  
 |  Chroma reader.
 |  
 |  Retrieve documents from existing persisted Chroma collections.
 |  
 |  Args:
 |      collection_name: Name of the peristed collection.
 |      persist_directory: Directory where the collection is persisted.
 |  
 |  Method resolution order:
 |      ChromaReader
 |      llama_index.readers.base.BaseReader
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, collection_name: str, persist_directory: str) -> None
 |      Initialize with parameters.
 |  
 |  load_data(self, query_vector: Any, limit: int = 10) -> Any
 |      Load data from Chroma.
 |      
 |      Args:
 |          query_vector (Any): Query
 |          limit (int): Number of results to return.
 |      
 |      Returns:
 |          List[Document]: A list of documents.
 |  
 |  -------------------------------------

In [89]:
chroma

In [88]:
# The chroma reader loads data from a persisted Chroma collection.
# This requires a collection name and a persist directory.
reader = ChromaReader(collection_name="specter_abstracts", persist_directory=collection)

ValidationError: 2 validation errors for Settings
persist_directory
  str type expected (type=type_error.str)
is_persistent
  extra fields not permitted (type=value_error.extra)

In [ ]:
### Generate Summary for Given Abstract
### LED Abstractive Text Summarization Model
def ledsummary_model(tokenizer, text, LEDmodel):
    summarize = "summarize: "
    encoding = tokenizer([summarize+text], truncation=True, return_tensors='pt' )
    # global_mask = torch.zeros_like(encoding)
    # global_mask[:,0] = 1
    output = LEDmodel.generate(encoding.input_ids,
                             num_beams=3,
                            no_repeat_ngram_size=2,
                              top_k=10,
                              top_p=80,
                              max_length=300,
                              min_length=50
                             )
    print(tokenizer.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))

In [ ]:

query_vector = [n1, n2, n3, ...]

documents = reader.load_data(
    collection_name="demo", query_vector=query_vector, limit=5)